In [1]:
!pip install -q "openai-agents[litellm]" python-dotenv

### OpenAI-Agent SDK 시작하기: 나만의 첫 AI 비서 만들기
이 튜토리얼에서는 OpenAI의 Assistants API를 사용하여, 대화의 맥락을 기억하고 주어진 도구를 활용하여 질문에 답변하는 AI 비서를 직접 만들어 봅니다.

### ✨ 학습 목표
OpenAI Assistants API의 핵심 개념(Assistant, Thread, Run)을 이해합니다.

도구 함수를 정의하고, OpenAI API가 요구하는 JSON 스키마로 변환하는 방법을 배웁니다.

상태를 가지는(Stateful) AI 비서와 대화하는 전체 과정을 코드로 구현합니다.

In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 API 키를 로드합니다.
load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    print("⚠️ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인해주세요!")
else:
    print("✅ API 키가 성공적으로 로드되었습니다.")

✅ API 키가 성공적으로 로드되었습니다.


### 단계 1: OpenAI Assistants API의 핵심 개념 이해하기
OpenAI Assistants는 세 가지 주요 구성 요소로 작동합니다.

#### Assistant (비서): 
특정 역할을 부여받은 AI입니다. "너는 날씨 전문가야"와 같은 지시사항(instructions), 사용할 수 있는 도구(tools), 그리고 LLM 모델을 설정하여 생성합니다. 한 번 만들어두면 계속 재사용할 수 있습니다.

#### Thread (대화 채널): 
사용자와 Assistant가 대화하는 개별 채널입니다. 각 Thread는 고유한 대화 기록을 가지므로, 여러 사용자가 동시에 대화해도 맥락이 섞이지 않습니다.

#### Run (실행): 
사용자가 Thread에 메시지를 남기면, Assistant가 그 메시지를 보고 작업을 수행하도록 지시하는 과정입니다. Assistant는 이 'Run' 과정에서 스스로 도구를 사용하거나 답변을 생성합니다.

In [3]:
from agents import Agent, Runner
import asyncio

# 1. Agent를 생성합니다.
agent = Agent(
    name="Haiku-bot",
    instructions="You are a helpful assistant that only responds in haikus.",
)

# 2. 비동기 함수를 정의합니다.
async def main():
    result = await Runner.run(agent, "AI 에이전트에 대한 한국어 시를 써줘")
    print("--- 에이전트의 최종 답변 ---")
    print(result)

await main()

--- 에이전트의 최종 답변 ---
RunResult:
- Last agent: Agent(name="Haiku-bot", ...)
- Final output (str):
    디지털 친구  
    무한한 지식 속에  
    길을 밝혀 줘
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [8]:
from agents import function_tool

@function_tool
def get_weather(city: str) -> str:
    """
    주어진 도시의 현재 날씨를 알려줍니다.
    
    Args:
        city (str): 날씨를 알고 싶은 도시 이름.
    """
    if city == "서울" or city == "Seoul":
        return "맑고 화창합니다. ☀️"
    else:
        return f"{city}의 날씨는 알 수 없습니다."

# 1. 새로운 도구를 장착한 에이전트를 생성합니다.
tool_agent = Agent(
    name="Weather-bot",
    instructions="You are a helpful agent that can check the weather.",
    tools=[get_weather], # 직접 만든 도구를 리스트에 담아 전달!
)

# 2. 에이전트에게 도구를 사용해야만 해결할 수 있는 질문을 던집니다.
async def main_tool():
    result = await Runner.run(tool_agent, "오늘 서울 날씨 어때?")
    print("--- 에이전트의 최종 답변 ---")
    print(result.final_output)

await main_tool()

--- 에이전트의 최종 답변 ---
오늘 서울은 맑고 화창한 날씨입니다. ☀️


In [10]:
from agents import WebSearchTool

# 1. WebSearchTool을 장착한 에이전트를 생성합니다.
search_agent = Agent(
    name="Search-bot",
    instructions="You are a helpful agent that searches the web.",
    tools=[WebSearchTool()],
)

# 2. 에이전트에게 웹 검색이 필요한 질문을 합니다.
async def main_search():
    result = await Runner.run(search_agent, "한국의 현재 인구는 몇 명인가요?")
    print("--- 에이전트의 최종 답변 ---")
    print(result.final_output)

await main_search()

--- 에이전트의 최종 답변 ---
2025년 현재 대한민국의 총인구는 약 5,200만 명으로 추정됩니다. ([seo.goover.ai](https://seo.goover.ai/report/202503/go-public-report-ko-9dd1a8f1-17bb-4014-8bb2-cd7ccd82df5a-0-0.html?utm_source=openai)) 그러나 저출산과 고령화로 인해 인구 감소가 지속될 것으로 예상되며, 통계청의 예측에 따르면 2072년에는 인구가 약 3,600만 명으로 줄어들 것으로 보입니다. ([seo.goover.ai](https://seo.goover.ai/report/202412/go-public-report-ko-b29fe432-1c74-435a-9a61-e54ad96ded71-0-0.html?utm_source=openai)) 


In [14]:
import asyncio
from agents import Agent, Runner, function_tool

# --- 1. 전문가 에이전트(Specialists) 생성 ---

# Weather Agent를 위한 도구
@function_tool
def get_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 알려줍니다."""
    if "서울" in city:
        return "맑고 화창합니다. ☀️"
    else:
        return f"{city}의 날씨는 알 수 없습니다."

# 날씨 전문가
weather_agent = Agent(
    name="Weather-Specialist",
    instructions="날씨에 관한 질문에 답변합니다.",
    tools=[get_weather],
)

# 수학 전문가
math_agent = Agent(
    name="Math-Specialist",
    instructions="수학 질문을 풀고 답변합니다.",
)


# --- 2. 라우터(Router) 에이전트 생성 ---
# as_tool()에 이름과 설명을 명확히 지정해줍니다.

ROUTER_INSTRUCTIONS = """
You are a router. Your job is to analyze the user's query and call the appropriate specialist tool to answer it.
"""

router_agent = Agent(
    name="Router",
    instructions=ROUTER_INSTRUCTIONS,
    tools=[
        weather_agent.as_tool(
            tool_name="Weather_Specialist",  # 도구 이름 지정
            tool_description="날씨에 관련된 질문을 처리하려면 이 전문가를 호출하세요." # 도구 설명 지정
        ),
        math_agent.as_tool(
            tool_name="Math_Specialist", # 도구 이름 지정
            tool_description="수학 문제나 계산이 필요하면 이 전문가를 호출하세요." # 도구 설명 지정
        ),
    ],
)


# --- 3. 에이전트 시스템 실행 ---

async def main():
    print("--- 날씨 질문 테스트 ---")
    weather_query = "오늘 서울 날씨 알려줘"
    weather_result = await Runner.run(router_agent, weather_query)
    print(f"질문: {weather_query}")
    print(f"답변: {weather_result.final_output}\n")

    print("--- 수학 질문 테스트 ---")
    math_query = "1234 * 5678은 얼마야?"
    math_result = await Runner.run(router_agent, math_query)
    print(f"질문: {math_query}")
    print(f"답변: {math_result.final_output}")

# 주피터 노트북 환경이므로 await로 실행
await main()

--- 날씨 질문 테스트 ---
질문: 오늘 서울 날씨 알려줘
답변: 오늘 서울의 날씨는 맑고 화창합니다. ☀️

--- 수학 질문 테스트 ---
질문: 1234 * 5678은 얼마야?
답변: 1234 * 5678은 7,006,652입니다.
